<a href="https://colab.research.google.com/github/VitalyGladyshev/gb_nlp/blob/main/%D0%93%D0%BB%D0%B0%D0%B4%D1%8B%D1%88%D0%B5%D0%B2%D0%92%D0%92_NLP_HW_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ДЗ 8 Гладышев ВВ

Берем отызывы за лето (из архива с материалами или предыдущего занятия)

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [2]:
path_otzv = "/content/отзывы за лето.xls"
data = pd.read_excel(path_otzv)
data.tail(3)

,Rating,Content,Date
20656,4,Доволен,2017-06-01
20657,1,"Песопаснасть, рут ни нужын",2017-06-01
20658,5,Сбербанк бомбовая компания на сегодняшний день...,2017-06-01


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20656 non-null  object
 2   Date     20659 non-null  object
dtypes: int64(1), object(2)
memory usage: 484.3+ KB


In [4]:
from datetime import datetime

# data["Year"] = [datetime.strptime(dt, '%Y-%m-%d').year for dt in data["Date"].values]
data["Month"] = [datetime.strptime(dt, '%Y-%m-%d').month for dt in data["Date"].values]
data["Day"] = [datetime.strptime(dt, '%Y-%m-%d').day for dt in data["Date"].values]
data.head()

,Rating,Content,Date,Month,Day
0,5,It just works!,2017-08-14,8,14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,8,14
2,5,Отлично все,2017-08-14,8,14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,8,14
4,5,"Очень удобно, работает быстро.",2017-08-14,8,14


## Задание

Провести сравнение RNN, LSTM, GRU

In [5]:
!pip install stop_words

  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=a964935539486ab188594c05e6618ad691adf438e91c615f0224928a3185b2ef
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words


In [6]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 32.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=840f88f11054eccff447010017add8668f4af61e3a89185a1a5cb4682b92aca1
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [7]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [8]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [9]:
data['Content_prep'] = data['Content'].apply(preprocess_text)
data.tail()

,Rating,Content,Date,Month,Day,Content_prep
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01,6,1,шляпас роот право бесполезный прогаразрабыв ох...
20655,5,Ок,2017-06-01,6,1,около
20656,4,Доволен,2017-06-01,6,1,довольный
20657,1,"Песопаснасть, рут ни нужын",2017-06-01,6,1,песопаснастя рута нужын
20658,5,Сбербанк бомбовая компания на сегодняшний день...,2017-06-01,6,1,сбербанк бомбовый компания сегодняшний лидер б...


In [10]:
train_corpus = " ".join(data['Content_prep'])

In [11]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens_tot = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
tokens_filtered = [word for word in tokens_tot if word.isalnum()]

In [13]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)

In [14]:
vocabulary = {v: i+100  for i, v in enumerate(dist) if dist[v]>1}

In [15]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return result[-maxlen:] + padding

In [16]:
data_embeded = np.asarray([text_to_sequence(text, 8) for text in data['Content_prep']], dtype=np.int32)
data_embeded = np.hstack(((data['Month'].values+50).reshape(data['Month'].values.shape[0], 1), data_embeded))
data_embeded = np.hstack(((data['Day'].values).reshape(data['Day'].values.shape[0], 1), data_embeded))
data_embeded

array([[  14,   58,  862, ...,    0,    0,    0],
       [  14,   58,  269, ...,  115,  649,  548],
       [  14,   58,  104, ...,    0,    0,    0],
       ...,
       [   1,   56,  140, ...,    0,    0,    0],
       [   1,   56,  123, ...,    0,    0,    0],
       [   1,   56,  117, ...,  116, 1372,    0]])

In [17]:
y = data['Rating'].values
train_x, valid_x, train_y, valid_y = train_test_split(data_embeded, y, stratify=y, random_state=43)

In [18]:
tr_data = train_x.reshape(train_x.shape[0], train_x.shape[1], 1)
vd_data = valid_x.reshape(valid_x.shape[0], valid_x.shape[1], 1)
tr_data.shape

(15494, 10, 1)

In [19]:
def to_one_hot(Y):
    n_col = np.amax(Y)
    binarized = np.zeros((len(Y), n_col))
    for i in range(len(Y)):
        binarized[i, Y[i]-1] = 1.
    return binarized

In [20]:
train_y_o = to_one_hot(train_y)
valid_y_o = to_one_hot(valid_y)
train_y_o

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

## Сравнение RNN, LSTM и GRU

In [21]:
import tensorflow as tf

# !pip install -q tensorflow-hub
# !pip install -q tensorflow-datasets
import tensorflow_hub as hub
# import tensorflow_datasets as tfds

import keras

from keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, SimpleRNN, GRU, Masking
from keras.utils import to_categorical
from tensorflow.keras import regularizers

from typing import List, Optional, Dict, Tuple

max_sequence_len = 10

In [22]:
def calculate_confidence_interval(scores: list, conf_interval: float = 0.95) -> Tuple[float]:
    """
    Вычисление доверительного интервала.

    Parameters
    ----------
    scores: List[float / int]
        Список с оценками изучаемой величины.

    conf_interval: float, optional, default = 0.95
        Уровень доверия для построения интервала.
        Опциональный параметр, по умолчанию, равен 0.95.

    Returns
    -------
    conf_interval: Tuple[float]
        Кортеж с границами доверительного интервала.

    """
    left_bound = np.percentile(
        scores, ((1 - conf_interval) / 2) * 100
    )
    right_bound = np.percentile(
        scores, (conf_interval + ((1 - conf_interval) / 2)) * 100
    )

    return left_bound, right_bound

In [23]:
def net_gen(lay_name='GRU', func_act="tanh", units=10):
    model = Sequential()

    model.add(Embedding(np.max(data_embeded)+1, units, input_length=max_sequence_len))
    #model.add(Bidirectional(LSTM(50, return_sequences = True)))
    #model.add(Dropout(0.2))
    model.add(eval(lay_name)(units, activation=func_act))    # model.add(lay_name(units, activation=func_act))
    model.add(Dense(32, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dense(5, activation='softmax'))

    # компиляция модели
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),#
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.AUC()])

    return model

In [24]:
def net_iter(model, ep=10, batch_size=512):
    history = model.fit(tr_data,
                        train_y_o,
                        epochs=ep,
                        validation_data=(vd_data, valid_y_o),
                        verbose=0)
    return history.history

In [ ]:
summary_data = pd.DataFrame(data=[[0, 0, 0, 0, 0, 0, 0, 0]],
                            columns=['layers', 'func_act', 'unit_number', 'mean_train_accuracy', 'mean_val_accuracy', 'variance', 'l_b', 'r_b'])
cnt = 0
lay_name = ['SimpleRNN', 'LSTM', 'GRU']
f_act = ['tanh', 'sigmoid']    # 'relu',
unit_number = [32, 64]    # , 50
epochs = 15

for l_n in lay_name:
    for f_a in f_act:
        for u_n in unit_number:
            err_score = []
            err_val_score = []
            print(f"Модель: слой {l_n}, функция активации {f_a}, число блоков {u_n}, эпох {epochs}\n")
            model = net_gen(lay_name=l_n, func_act=f_a, units=u_n)
            model.summary()
            for i in range(5):
                print(f"\tИтерация: {i+1}")
                hist = net_iter(model, ep=epochs)
                err_score.append(hist[list(hist.keys())[1]][-1])
                err_val_score.append(hist[list(hist.keys())[3]][-1])

            vr = np.var(err_val_score)
            l_b, r_b = calculate_confidence_interval(err_val_score)
            mean_err_score, mean_err_val_score = np.mean(err_score), np.mean(err_val_score)
            print(f"Ошибки на train. Средняя: {mean_err_score} список: {err_score}")
            print(f"Ошибки на valid. Средняя: {mean_err_val_score} список: {err_val_score}")
            print(f"Дисперсия ошибки на valid: {vr}")
            print(f"Доверительный интервал: {l_b} - {r_b}\n\n")
            summary_data.loc[cnt, ['layers', 'func_act', 'unit_number', 'mean_train_score',
                                    'mean_val_score', 'variance', 'l_b', 'r_b']] = [l_n, f_a, u_n,
                                            np.mean(err_score), np.mean(err_val_score), vr, l_b, r_b]
            cnt += 1

Модель: слой SimpleRNN, функция активации tanh, число блоков 32, эпох 15

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 32)            413600    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                2080      
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 165       
Total params: 416,901
Trainable params: 416,901
Non-trainable params: 0
_________________________________________________________________
	Итерация: 1
	Итерация: 2
	Итерация: 3
	Итерация: 4
	Итерация: 5
Ошибки на train. Средняя: 0.9024798274040222 список: [0.8820685744285583, 0.8837203979492188, 0.9104104638099

In [ ]:
summary_data.drop(columns=["mean_train_accuracy", "mean_val_accuracy"])

,layers,func_act,unit_number,variance,l_b,r_b,mean_train_score,mean_val_score
0,SimpleRNN,tanh,32.0,0.000017,0.865822,0.876553,0.902480,0.870917
1,SimpleRNN,tanh,64.0,0.000516,0.816312,0.866609,0.842756,0.834633
2,SimpleRNN,sigmoid,32.0,0.000000,0.816312,0.816312,0.816260,0.816312
3,SimpleRNN,sigmoid,64.0,0.000000,0.816312,0.816312,0.816260,0.816312
4,LSTM,tanh,32.0,0.000007,0.868809,0.875856,0.908089,0.871259
5,LSTM,tanh,64.0,0.000006,0.866739,0.873474,0.909013,0.869705
6,LSTM,sigmoid,32.0,0.000000,0.816312,0.816312,0.816260,0.816312
7,LSTM,sigmoid,64.0,0.000000,0.816312,0.816312,0.816260,0.816312
8,GRU,tanh,32.0,0.000019,0.864886,0.875691,0.904983,0.869554
9,GRU,tanh,64.0,0.000028,0.862530,0.877291,0.908321,0.868601


In [ ]:
summary_data.drop(columns=["mean_train_accuracy", "mean_val_accuracy"]).sort_values('mean_val_score', ascending=False)

,layers,func_act,unit_number,variance,l_b,r_b,mean_train_score,mean_val_score
4,LSTM,tanh,32.0,0.000007,0.868809,0.875856,0.908089,0.871259
0,SimpleRNN,tanh,32.0,0.000017,0.865822,0.876553,0.902480,0.870917
5,LSTM,tanh,64.0,0.000006,0.866739,0.873474,0.909013,0.869705
8,GRU,tanh,32.0,0.000019,0.864886,0.875691,0.904983,0.869554
9,GRU,tanh,64.0,0.000028,0.862530,0.877291,0.908321,0.868601
1,SimpleRNN,tanh,64.0,0.000516,0.816312,0.866609,0.842756,0.834633
2,SimpleRNN,sigmoid,32.0,0.000000,0.816312,0.816312,0.816260,0.816312
3,SimpleRNN,sigmoid,64.0,0.000000,0.816312,0.816312,0.816260,0.816312
6,LSTM,sigmoid,32.0,0.000000,0.816312,0.816312,0.816260,0.816312
7,LSTM,sigmoid,64.0,0.000000,0.816312,0.816312,0.816260,0.816312


## Пробуем лучший вариант

In [27]:
model = Sequential()

model.add(Embedding(np.max(data_embeded)+1, 32, input_length=max_sequence_len))
# model.add(Bidirectional(LSTM(32, return_sequences = False)))
model.add(LSTM(32, activation='tanh', input_shape=(tr_data.shape[1], 1))) # model.add(LSTM(32, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(5, activation='softmax'))

# компиляция модели
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),#
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
            metrics=[tf.keras.metrics.AUC()])

In [28]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 10, 32)            146432    
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_4 (Dense)             (None, 32)                1056      
                                                                 
 dense_5 (Dense)             (None, 5)                 165       
                                                                 
Total params: 155973 (609.27 KB)
Trainable params: 155973 (609.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [30]:
hist = model.fit(tr_data,
                 train_y_o,
                 batch_size=512,
                 epochs=1000,
                 validation_data=(vd_data, valid_y_o),
                 callbacks=[callback],
                 verbose=1)

Epoch 1/1000
31/31 [==============================] - 9s 151ms/step - loss: 1.8457 - auc_2: 0.6258 - val_loss: 1.4123 - val_auc_2: 0.8654
Epoch 2/1000
31/31 [==============================] - 3s 106ms/step - loss: 1.2125 - auc_2: 0.8663 - val_loss: 1.0975 - val_auc_2: 0.9100
Epoch 3/1000
31/31 [==============================] - 3s 102ms/step - loss: 1.0221 - auc_2: 0.9135 - val_loss: 0.9632 - val_auc_2: 0.9232
Epoch 4/1000
31/31 [==============================] - 2s 61ms/step - loss: 0.9125 - auc_2: 0.9276 - val_loss: 0.8765 - val_auc_2: 0.9317
Epoch 5/1000
31/31 [==============================] - 2s 56ms/step - loss: 0.8312 - auc_2: 0.9359 - val_loss: 0.8260 - val_auc_2: 0.9350
Epoch 6/1000
31/31 [==============================] - 2s 62ms/step - loss: 0.7816 - auc_2: 0.9405 - val_loss: 0.7981 - val_auc_2: 0.9355
Epoch 7/1000
31/31 [==============================] - 1s 28ms/step - loss: 0.7452 - auc_2: 0.9423 - val_loss: 0.7809 - val_auc_2: 0.9349
Epoch 8/1000
31/31 [==================

## Пробуем увеличить длину последовательности

In [58]:
data_embeded = np.asarray([text_to_sequence(text, 18) for text in data['Content_prep']], dtype=np.int32)
data_embeded = np.hstack(((data['Month'].values+50).reshape(data['Month'].values.shape[0], 1), data_embeded))
data_embeded = np.hstack(((data['Day'].values).reshape(data['Day'].values.shape[0], 1), data_embeded))
data_embeded

array([[ 14,  58, 862, ...,   0,   0,   0],
       [ 14,  58, 202, ...,   0,   0,   0],
       [ 14,  58, 104, ...,   0,   0,   0],
       ...,
       [  1,  56, 140, ...,   0,   0,   0],
       [  1,  56, 123, ...,   0,   0,   0],
       [  1,  56, 117, ...,   0,   0,   0]])

In [59]:
y = data['Rating'].values
train_x, valid_x, train_y, valid_y = train_test_split(data_embeded, y, stratify=y, random_state=43)

In [60]:
tr_data = train_x.reshape(train_x.shape[0], train_x.shape[1], 1)
vd_data = valid_x.reshape(valid_x.shape[0], valid_x.shape[1], 1)
tr_data.shape

(15494, 20, 1)

In [61]:
model = Sequential()

model.add(Embedding(np.max(data_embeded)+1, 32, input_length=20))
# model.add(Bidirectional(LSTM(32, return_sequences = False)))
model.add(LSTM(32, activation='tanh')) # model.add(LSTM(32, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(5, activation='softmax'))

# компиляция модели
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),#
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
            metrics=[tf.keras.metrics.AUC()])

In [62]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 20, 32)            146464    
                                                                 
 lstm_3 (LSTM)               (None, 32)                8320      
                                                                 
 dropout_8 (Dropout)         (None, 32)                0         
                                                                 
 dense_14 (Dense)            (None, 32)                1056      
                                                                 
 dense_15 (Dense)            (None, 5)                 165       
                                                                 
Total params: 156005 (609.39 KB)
Trainable params: 156005 (609.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [63]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

In [64]:
hist = model.fit(tr_data,
                 train_y_o,
                 batch_size=512,
                 epochs=1000,
                 validation_data=(vd_data, valid_y_o),
                 callbacks=[callback],
                 shuffle=True,
                 verbose=1)

Epoch 1/1000


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


31/31 [==============================] - 6s 138ms/step - loss: 1.5754 - auc_7: 0.7956 - val_loss: 1.2408 - val_auc_7: 0.8566
Epoch 2/1000
31/31 [==============================] - 3s 84ms/step - loss: 1.1609 - auc_7: 0.8817 - val_loss: 1.0744 - val_auc_7: 0.9102
Epoch 3/1000
31/31 [==============================] - 2s 54ms/step - loss: 0.9881 - auc_7: 0.9218 - val_loss: 0.9039 - val_auc_7: 0.9309
Epoch 4/1000
31/31 [==============================] - 2s 67ms/step - loss: 0.8576 - auc_7: 0.9328 - val_loss: 0.8425 - val_auc_7: 0.9355
Epoch 5/1000
31/31 [==============================] - 1s 49ms/step - loss: 0.7968 - auc_7: 0.9389 - val_loss: 0.8138 - val_auc_7: 0.9344
Epoch 6/1000
31/31 [==============================] - 1s 36ms/step - loss: 0.7566 - auc_7: 0.9412 - val_loss: 0.7856 - val_auc_7: 0.9350
Epoch 7/1000
31/31 [==============================] - 2s 51ms/step - loss: 0.7267 - auc_7: 0.9428 - val_loss: 0.7685 - val_auc_7: 0.9353
Epoch 8/1000
31/31 [==============================] -

## Переделываем эмбеддинг

In [38]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [39]:
tokenizer = Tokenizer(num_words=None,
                      filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                      lower=False,
                      split=' ')
tokenizer.fit_on_texts(data['Content_prep'])

In [40]:
y = data['Rating'].values
train_x, valid_x, train_y, valid_y = train_test_split(data['Content_prep'], y, stratify=y, random_state=43)

In [41]:
sequences_train = tokenizer.texts_to_sequences(train_x.values)
sequences_val = tokenizer.texts_to_sequences(valid_x.values)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in data['Content_prep'].values])

tr_data = pad_sequences(sequences_train, maxlen=training_length)
vd_data = pad_sequences(sequences_val, maxlen=training_length)

In [42]:
def net_gen(lay_name='GRU', func_act="tanh", units=10):
    model = Sequential()

    model.add(Embedding(input_dim=word_count,
                        input_length=training_length,
                        output_dim=30,
                        trainable=True,
                        mask_zero=True))

    model.add(Masking(mask_value=0.0))
    model.add(eval(lay_name)(units, activation=func_act))
    model.add(Dense(32, activation='sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))

    # компиляция модели
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),#
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.AUC()])

    return model

In [43]:
def net_iter(model, ep=10, batch_size=512):
    history = model.fit(tr_data,
                        train_y_o,
                        epochs=ep,
                        validation_data=(vd_data, valid_y_o),
                        verbose=1)
    return history.history

In [ ]:
summary_data = pd.DataFrame(data=[[0, 0, 0, 0, 0, 0, 0, 0]],
                            columns=['layers', 'func_act', 'unit_number', 'mean_train_score', 'mean_val_score', 'variance', 'l_b', 'r_b'])
cnt = 0
lay_name = ['SimpleRNN', 'LSTM', 'GRU']
f_act = ['tanh', 'sigmoid']    # 'relu',
unit_number = [32, 64]    # , 50
epochs = 5

for l_n in lay_name:
    for f_a in f_act:
        for u_n in unit_number:
            err_score = []
            err_val_score = []
            print(f"Модель: слой {l_n}, функция активации {f_a}, число блоков {u_n}, эпох {epochs}\n")
            model = net_gen(lay_name=l_n, func_act=f_a, units=u_n)
            model.summary()
            for i in range(3):
                print(f"\tИтерация: {i+1}")
                hist = net_iter(model, ep=epochs)
                err_score.append(hist[list(hist.keys())[1]][-1])
                err_val_score.append(hist[list(hist.keys())[3]][-1])

            vr = np.var(err_val_score)
            l_b, r_b = calculate_confidence_interval(err_val_score)
            mean_err_score, mean_err_val_score = np.mean(err_score), np.mean(err_val_score)
            print(f"Ошибки на train. Средняя: {mean_err_score} список: {err_score}")
            print(f"Ошибки на valid. Средняя: {mean_err_val_score} список: {err_val_score}")
            print(f"Дисперсия ошибки на valid: {vr}")
            print(f"Доверительный интервал: {l_b} - {r_b}\n\n")
            summary_data.loc[cnt, ['layers', 'func_act', 'unit_number', 'mean_train_score',
                                    'mean_val_score', 'variance', 'l_b', 'r_b']] = [l_n, f_a, u_n,
                                            np.mean(err_score), np.mean(err_val_score), vr, l_b, r_b]
            cnt += 1

Модель: слой SimpleRNN, функция активации tanh, число блоков 32, эпох 5

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 132, 30)           404580    
_________________________________________________________________
masking_7 (Masking)          (None, 132, 30)           0         
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 32)                2016      
_________________________________________________________________
dense_14 (Dense)             (None, 32)                1056      
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 5)                 165       
Total params: 407,817
Trainable params: 407,817

In [ ]:
summary_data

,layers,func_act,unit_number,mean_train_score,mean_val_score,variance,l_b,r_b
0,SimpleRNN,tanh,32.0,0.883360,0.870790,0.000365,0.856165,0.895803
1,SimpleRNN,tanh,64.0,0.816323,0.816312,0.000000,0.816312,0.816312
2,SimpleRNN,sigmoid,32.0,0.816299,0.816312,0.000000,0.816312,0.816312
3,SimpleRNN,sigmoid,64.0,0.816293,0.816312,0.000000,0.816312,0.816312
4,LSTM,tanh,32.0,0.816361,0.816312,0.000000,0.816312,0.816312
5,LSTM,tanh,64.0,0.816189,0.816312,0.000000,0.816312,0.816312
6,LSTM,sigmoid,32.0,0.816319,0.816312,0.000000,0.816312,0.816312
7,LSTM,sigmoid,64.0,0.816290,0.816312,0.000000,0.816312,0.816312
8,GRU,tanh,32.0,0.816226,0.816312,0.000000,0.816312,0.816312
9,GRU,tanh,64.0,0.816331,0.816312,0.000000,0.816312,0.816312


In [ ]:
summary_data.sort_values('mean_val_score', ascending=False)

,layers,func_act,unit_number,mean_train_score,mean_val_score,variance,l_b,r_b
0,SimpleRNN,tanh,32.0,0.883360,0.870790,0.000365,0.856165,0.895803
1,SimpleRNN,tanh,64.0,0.816323,0.816312,0.000000,0.816312,0.816312
2,SimpleRNN,sigmoid,32.0,0.816299,0.816312,0.000000,0.816312,0.816312
3,SimpleRNN,sigmoid,64.0,0.816293,0.816312,0.000000,0.816312,0.816312
4,LSTM,tanh,32.0,0.816361,0.816312,0.000000,0.816312,0.816312
5,LSTM,tanh,64.0,0.816189,0.816312,0.000000,0.816312,0.816312
6,LSTM,sigmoid,32.0,0.816319,0.816312,0.000000,0.816312,0.816312
7,LSTM,sigmoid,64.0,0.816290,0.816312,0.000000,0.816312,0.816312
8,GRU,tanh,32.0,0.816226,0.816312,0.000000,0.816312,0.816312
9,GRU,tanh,64.0,0.816331,0.816312,0.000000,0.816312,0.816312


## Пробуем стек

In [50]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
                    input_length=training_length,
                    output_dim=30,
                    # mask_zero=True,
                    trainable=True))

# model.add(Masking(mask_value=0.0))
model.add(SimpleRNN(32, return_sequences = True))   # , activation="tanh"
model.add(Dropout(0.2))
model.add(SimpleRNN(32))   # , activation="tanh"
model.add(Dense(32, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

# компиляция модели
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),#
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
            metrics=[tf.keras.metrics.AUC()])

In [51]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 132, 30)           404580    
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 132, 32)           2016      
                                                                 
 dropout_5 (Dropout)         (None, 132, 32)           0         
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense_10 (Dense)            (None, 32)                1056      
                                                                 
 dropout_6 (Dropout)         (None, 32)                0         
                                                                 
 dense_11 (Dense)            (None, 5)                

In [52]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

In [53]:
hist = model.fit(tr_data,
                 train_y_o,
                 batch_size=512,
                 epochs=1000,
                 validation_data=(vd_data, valid_y_o),
                 callbacks=[callback],
                 shuffle=True,
                 verbose=1)

Epoch 1/1000
31/31 [==============================] - 15s 398ms/step - loss: 1.3875 - auc_5: 0.7402 - val_loss: 0.9942 - val_auc_5: 0.8748
Epoch 2/1000
31/31 [==============================] - 11s 342ms/step - loss: 1.0532 - auc_5: 0.8465 - val_loss: 0.8589 - val_auc_5: 0.9071
Epoch 3/1000
31/31 [==============================] - 9s 296ms/step - loss: 0.9185 - auc_5: 0.8850 - val_loss: 0.7832 - val_auc_5: 0.9238
Epoch 4/1000
31/31 [==============================] - 9s 295ms/step - loss: 0.8218 - auc_5: 0.9093 - val_loss: 0.7257 - val_auc_5: 0.9340
Epoch 5/1000
31/31 [==============================] - 9s 289ms/step - loss: 0.7515 - auc_5: 0.9250 - val_loss: 0.7062 - val_auc_5: 0.9367
Epoch 6/1000
31/31 [==============================] - 9s 305ms/step - loss: 0.6951 - auc_5: 0.9365 - val_loss: 0.6924 - val_auc_5: 0.9377
Epoch 7/1000
31/31 [==============================] - 8s 260ms/step - loss: 0.6565 - auc_5: 0.9436 - val_loss: 0.7066 - val_auc_5: 0.9365
Epoch 8/1000
31/31 [============

In [54]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

# компиляция модели
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),#
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
            metrics=[tf.keras.metrics.AUC()])

In [55]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 132, 30)           404580    
                                                                 
 masking (Masking)           (None, 132, 30)           0         
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 64)                6080      
                                                                 
 dense_12 (Dense)            (None, 64)                4160      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_13 (Dense)            (None, 5)                 325       
                                                                 
Total params: 415145 (1.58 MB)
Trainable params: 41514

In [56]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

In [57]:
hist = model.fit(tr_data,
                 train_y_o,
                 batch_size=512,
                 epochs=1000,
                 validation_data=(vd_data, valid_y_o),
                 callbacks=[callback],
                 shuffle=True,
                 verbose=1)

Epoch 1/1000
31/31 [==============================] - 12s 334ms/step - loss: 1.1103 - auc_6: 0.8304 - val_loss: 0.9015 - val_auc_6: 0.9046
Epoch 2/1000
31/31 [==============================] - 9s 298ms/step - loss: 0.8874 - auc_6: 0.8922 - val_loss: 0.7980 - val_auc_6: 0.9217
Epoch 3/1000
31/31 [==============================] - 8s 260ms/step - loss: 0.7690 - auc_6: 0.9220 - val_loss: 0.7311 - val_auc_6: 0.9322
Epoch 4/1000
31/31 [==============================] - 9s 283ms/step - loss: 0.6817 - auc_6: 0.9398 - val_loss: 0.6916 - val_auc_6: 0.9380
Epoch 5/1000
31/31 [==============================] - 9s 287ms/step - loss: 0.6150 - auc_6: 0.9513 - val_loss: 0.6736 - val_auc_6: 0.9406
Epoch 6/1000
31/31 [==============================] - 7s 242ms/step - loss: 0.5569 - auc_6: 0.9602 - val_loss: 0.6825 - val_auc_6: 0.9397
Epoch 7/1000
31/31 [==============================] - 8s 241ms/step - loss: 0.5050 - auc_6: 0.9673 - val_loss: 0.6934 - val_auc_6: 0.9382
Epoch 8/1000
31/31 [=============